In [1]:
import sys
sys.path.append('../scripts')
import polars as pl
from helpers import load_master_dataset, get_transactions, print_transaction_log

# Load master dataset and filter for our session of interest
source_file = "orig_adc_record.6"
df = load_master_dataset('../../usb_master_dataset.parquet')
session_df = df.filter(pl.col('source_file') == source_file)

print(f"Analyzing session: '{source_file}' ({len(session_df)} packets)")


✅ Loaded 11,514 USB packets from ../../usb_master_dataset.parquet
Analyzing session: 'orig_adc_record.6' (420 packets)


In [2]:
# Process the session data into a chronological list of transactions
transactions = get_transactions(session_df)

# Display the transaction log
print_transaction_log(transactions, limit=40)


Found 210 logical transactions. Displaying first 40:
--------------------------------------------------------------------------------------------------------------
#   Timestamp (s) Duration (ms) | Type            | Submit Details                 | Complete Details
--------------------------------------------------------------------------------------------------------------
1   0.000000             0.213 | Control Setup   | D->H   0b -             ...    | D->H  18b -             ...
2   0.000220             0.163 | Control Setup   | D->H   0b -             ...    | D->H   9b -             ...
3   0.000386             0.419 | Control Setup   | D->H   0b -             ...    | D->H 130b -             ...
4   2.154407             0.196 | Control Setup   | D->H   0b -             ...    | D->H  22b -             ...
5   2.155386             0.189 | Control Setup   | D->H   0b -             ...    | D->H  30b -             ...
6   2.158666             0.188 | Control Setup   | D->H   0b - 

In [3]:
from collections import Counter

# --- Transaction Statistics ---
print('--- TRANSACTION STATISTICS ---')

# 1. Count by Type
type_counts = Counter(tx['type'] for tx in transactions)
print('\\nTransaction Counts by Type:')
for tx_type, count in type_counts.most_common():
    percentage = (count / len(transactions)) * 100
    print(f'  - {tx_type:<15}: {count:>4} transactions ({percentage:.1f}%)')

# 2. Duration Statistics by Type
duration_stats = {}
for tx in transactions:
    tx_type = tx['type']
    if tx_type not in duration_stats:
        duration_stats[tx_type] = []
    duration_stats[tx_type].append(tx['duration_ms'])

print('\\nAverage Duration by Type (ms):')
for tx_type, durations in sorted(duration_stats.items()):
    avg_duration = sum(durations) / len(durations)
    min_duration = min(durations)
    max_duration = max(durations)
    print(f'  - {tx_type:<15}: Avg: {avg_duration:8.3f}ms | Min: {min_duration:8.3f}ms | Max: {max_duration:8.3f}ms')
    
# 3. Data Transfer Statistics
print('\\nData Transfer by Type:')
data_transfer_stats = {}
for tx in transactions:
    tx_type = tx['type']
    if tx_type not in data_transfer_stats:
        data_transfer_stats[tx_type] = {'submit_bytes': 0, 'complete_bytes': 0, 'count': 0}
    
    data_transfer_stats[tx_type]['submit_bytes'] += tx['submit']['data_length'][0]
    data_transfer_stats[tx_type]['complete_bytes'] += tx['complete']['data_length'][0]
    data_transfer_stats[tx_type]['count'] += 1

for tx_type, stats in sorted(data_transfer_stats.items()):
    total_bytes = stats['submit_bytes'] + stats['complete_bytes']
    avg_bytes = total_bytes / stats['count']
    print(f'  - {tx_type:<15}: Total: {total_bytes:>5} bytes | Avg per Tx: {avg_bytes:6.1f} bytes')


--- TRANSACTION STATISTICS ---
\nTransaction Counts by Type:
  - Device Data    :  100 transactions (47.6%)
  - Host Command   :   95 transactions (45.2%)
  - Control Setup  :   13 transactions (6.2%)
  - Control/ACK    :    2 transactions (1.0%)
\nAverage Duration by Type (ms):
  - Control Setup  : Avg:    0.212ms | Min:    0.163ms | Max:    0.419ms
  - Control/ACK    : Avg:   78.655ms | Min:   74.941ms | Max:   82.369ms
  - Device Data    : Avg:  157.882ms | Min:    0.074ms | Max:  237.636ms
  - Host Command   : Avg:    0.111ms | Min:    0.068ms | Max:    0.627ms
\nData Transfer by Type:
  - Control Setup  : Total:   339 bytes | Avg per Tx:   26.1 bytes
  - Control/ACK    : Total:     0 bytes | Avg per Tx:    0.0 bytes
  - Device Data    : Total:  4872 bytes | Avg per Tx:   48.7 bytes
  - Host Command   : Total:   540 bytes | Avg per Tx:    5.7 bytes
